## Напишем алгоритм для работы с видеопотоком. Vol.3

### Загрузим библиотеки: 

In [ ]:
#!pip install --upgrade tensorflow
#pip install plaidml

In [17]:
import os
import time
import sys
import tensorflow as tf
import keras
import cv2
from math import sqrt
import numpy as np
from PIL import Image
from scipy.spatial import distance
import datetime
import logging as log
from cv2 import CascadeClassifier
from keras import models
from matplotlib import pyplot as plt
from keras.models import load_model
import tensorflow as tf
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN

In [ ]:
!pip freeze > requirements.txt

In [18]:
# загрузим обученную нами ранее модель:
model = tf.keras.models.load_model('model_mask.hdf5',
                                   custom_objects=None, compile=True, options=None)
#model = models.load_model('model_mask.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.40_224 (Functi (None, 4, 4, 1792)        4363712   
_________________________________________________________________
flatten (Flatten)            (None, 28672)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               7340288   
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0

Наш алгоритм будет принимать видеопоток, разбивать его на кадры, находить лицо, 
анализировать через нашу обученную модель, давать оценку на наличие МАСКИ на лице у человека, выводить об этом информацию в виде надписи, также алгоритм считает кол-во лиц в кадре, и в случае нарушения (превышен лимит лиц 3+) - будет об этом сигнализировать.
Также алгоритм считает расстояние между лицами и сравнивает с заданной дистанцией. 
Если она превышена, то будет сигнал в виде сообщения.
Также алгоритм делает сохранение полученного и обработанного видео в файл для возможности дальнейшего анализа.
Случаи нарушения заданных требований (кол-во лиц, "Без маски", превышение соц. дистанции - будут логироваться в файл с датой нарушения, а также фотофиксация моментов нарушения, для дальнейшего анализа ситуации.


### 1.Вариант работы с open CV Haar cascade:

Видеопоток можно получить как с web-camera notebook, так и через IP-camera, а также просто видеофайл. 

In [12]:
MIN_DISTANCE = 200  # distance between people for control
mask_label = {0: 'NO MASK', 1: 'MASK'}
dist_label = {0: (0, 255, 0), 1: (0, 0, 255)}  # color for distance ok\bad
color_label = {0: (0, 0, 255), 1: (0, 255, 0), 2: (
    255, 0, 0), 3: (0, 0, 0), 4: (255, 255, 255)}
green = (0, 255, 0)
red = (255, 0, 0)
blue = (0, 0, 255)
counter = 0
log.basicConfig(filename='webcam2.log', level=log.INFO) #log start

video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture("VIDEO0102.mp4") # for file
# video_capture ='http://admin:Ebmacs8485867@'+'192.168.100.23'+':81/web/admin.html' # for online
# cap=VideoCapture("rtsp://user_name:password@IP_address:port_number")
video_capture.set(3, 640)
video_capture.set(4, 480)
time.sleep(2.0)
faceCascade = CascadeClassifier('haarcascade_frontalface_default.xml')
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output4.mp4', fourcc, 5.0, (640, 480)) # save video file

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10,
                                         minSize=(20, 20), flags=cv2.CASCADE_SCALE_IMAGE)
    out_img = frame
    time_now = datetime.datetime.now().replace(microsecond=0)  # time now show
    cv2.putText(out_img, str('People count, mask & distance controller WEB @by Pawel_MTW'),
                (100, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(out_img, str(time_now), (100, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(out_img, f"People: {len(faces)}", (100, 110), # count faces info show
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)


    if len(faces) >= 2:
        label = [0 for i in range(len(faces))]
        for i in range(len(faces)-1):
            for j in range(i+1, len(faces)):
                dist = distance.euclidean(faces[i][:2], faces[j][:2])
                if dist < MIN_DISTANCE:
                    label[i] = 1
                    label[j] = 1
                    counter += 1
                    cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo bad distance
                    log.info("bad distance"+str(time_now)) # log bad distance

        for i in range(len(faces)):
            (x, y, w, h) = faces[i]
            crop = out_img[y:y+h, x:x+w]
            crop = cv2.resize(crop, (120, 120))
            crop = np.reshape(crop, [1, 120, 120, 3])/255.0
            mask_result = model.predict(crop)

            if mask_result > 0.50:  # mask 
                cv2.rectangle(out_img, (x, y), (x+w, y+h),
                              dist_label[label[i]], 3)
                cv2.rectangle(out_img, (x, y-30), (x+w, y),
                              dist_label[label[i]], -1)
                cv2.putText(
                    out_img, mask_label[1], (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if mask_result < 0.50: # no_mask 
                cv2.rectangle(out_img, (x, y), (x+w, y+h),
                              dist_label[label[i]], 3)
                cv2.rectangle(out_img, (x, y-30), (x+w, y),
                              dist_label[label[i]], -1)
                cv2.putText(
                    out_img, mask_label[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                counter += 1
                cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo no mask case
                log.info("no mask at"+str(time_now)) # log no mask case

    else:
        for i in range(len(faces)):
            (x, y, w, h) = faces[i]
            crop = out_img[y:y+h, x:x+w]
            crop = cv2.resize(crop, (120, 120))
            crop = np.reshape(crop, [1, 120, 120, 3])/255.0
            mask_result = model.predict(crop)

            if mask_result > 0.50: # mask 
                cv2.rectangle(out_img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.rectangle(out_img, (x, y-50), (x+w, y), (0, 255, 0), -1)
                cv2.putText(
                    out_img, mask_label[1], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if mask_result < 0.50: # no_mask 
                cv2.rectangle(out_img, (x, y), (x+w, y+h), (255, 0, 0), 3)
                cv2.rectangle(out_img, (x, y-50), (x+w, y), (255, 0, 0), -1)
                cv2.putText(
                    out_img, mask_label[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                counter += 1
                cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo no mask case
                log.info("no mask at"+str(time_now)) # log no mask case

# Display alarm if number of people 3+:
    if len(faces) >= 3:
        cv2.putText(out_img, str("Alarm: 3+ people"), (100, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        counter += 1
        cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo people 3+
        log.info("people: "+str(len(faces))+" at "+str(time_now)) # log no people 3+

# Display the resulting frame
    cv2.imshow('People count, mask & distance controller @by Pawel_MTW', out_img)
    out.write(out_img)
    if cv2.waitKey(100) & 0xFF == ord('q'):  # cv2.waitKey - время задержки кадра
        break
# When everything is done, release the capture
video_capture.release()
out.release()
cv2.destroyAllWindows()

### 2.Вариант работы с MTCNN:

In [14]:
import plaidml.keras
import os
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
import keras.backend as K

In [34]:
# работает 100%

MIN_DISTANCE = 200  # distance between people for control
mask_label = {0: 'NO MASK', 1: 'MASK'}
dist_label = {0: (0, 255, 0), 1: (0, 0, 255)}  # color for distance ok\bad
green = (0, 255, 0)
red = (255, 0, 0)
blue = (0, 0, 255)
counter = 0
log.basicConfig(filename='webcam.log',level=log.INFO)
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture("VIDEO0102.mp4") # for file upload
# video_capture ='http://admin:Ebmacs8485867@'+'192.168.100.23'+':81/web/admin.html' # for online
# cap=VideoCapture("rtsp://user_name:password@IP_address:port_number")
video_capture.set(3,640)
video_capture.set(4,480)
time.sleep(2.0)
#fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
#out = cv2.VideoWriter('output5.mp4', fourcc, 5.0, (640,480)
out = cv2.VideoWriter('output.avi', fourcc, 1.0, (640,480))

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    detector = MTCNN()
    faces = detector.detect_faces(frame)
    out_img = frame
    time_now = datetime.datetime.now().replace(microsecond=0) # show time info 
    cv2.putText(out_img, str('People count, mask & distance controller WEB @by Pawel_MTW'),
                (100, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(out_img, str(time_now), (100, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(out_img, f"People: {len(faces)}", (100, 110), # show people count info 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    if len(faces) >= 2:
        label = [0 for i in range(len(faces))]
        for i in range(len(faces)-1):
            for j in range(i+1, len(faces)):
                dist = distance.euclidean(
                    faces[i]["box"][:2], faces[j]["box"][:2])
                if dist < MIN_DISTANCE:
                    label[i] = 1
                    label[j] = 1
                    counter += 1
                    cv2.imwrite(f"alarm{counter}.jpg", out_img)# save photo bad distance
                    log.info("bad distance"+str(time_now))  # log bad distance

        for i in range(len(faces)):
            #(x,y,w,h) = faces[i]
            (x, y, w, h) = faces[i]["box"]
            crop = out_img[y:y+h, x:x+w]
            crop = cv2.resize(crop, (120, 120))
            crop = np.reshape(crop, [1, 120, 120, 3])/255.0
            mask_result = model.predict(crop)

            if mask_result > 0.55:
                cv2.rectangle(out_img, (x, y), (x+w, y+h),
                              dist_label[label[i]], 3)
                cv2.rectangle(out_img, (x, y-30), (x+w, y),
                              dist_label[label[i]], -1)
                cv2.putText(
                    out_img, mask_label[1], (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if mask_result < 0.45:
                cv2.rectangle(out_img, (x, y), (x+w, y+h),
                              dist_label[label[i]], 3)
                cv2.rectangle(out_img, (x, y-30), (x+w, y),
                              dist_label[label[i]], -1)
                cv2.putText(
                    out_img, mask_label[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                counter += 1
                cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo no mask case
                log.info("no mask at"+str(time_now))  # log no mask case

    else:
        for i in range(len(faces)):
            (x, y, w, h) = faces[i]["box"]
            crop = out_img[y:y+h, x:x+w]
            crop = cv2.resize(crop, (120, 120))
            crop = np.reshape(crop, [1, 120, 120, 3])/255.0
            mask_result = model.predict(crop)

            if mask_result > 0.55:
                cv2.rectangle(out_img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.rectangle(out_img, (x, y-50), (x+w, y), (0, 255, 0), -1)
                cv2.putText(
                    out_img, mask_label[1], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if mask_result < 0.45:
                cv2.rectangle(out_img, (x, y), (x+w, y+h), (255, 0, 0), 3)
                cv2.rectangle(out_img, (x, y-50), (x+w, y), (255, 0, 0), -1)
                cv2.putText(
                    out_img, mask_label[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                counter += 1
                cv2.imwrite(f"alarm{counter}.jpg", out_img) # save photo no mask case
                log.info("no mask at"+str(time_now))  # log no mask case

# Display alarm if number of people 3+:
    if len(faces) >= 3:
        cv2.putText(out_img, str("Alarm: 3+ people"), (100, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        counter += 1
        cv2.imwrite(f"alarm{counter}.jpg", out_img)  # save  photo people 3+ case
        log.info("people: "+len(faces)+" at " +
                 str(time_now))  # log people 3+ case

# Display the resulting frame
    cv2.imshow('People count, mask & distance controller @by Pawel_MTW', out_img)
    out.write(out_img)
    if cv2.waitKey(100) & 0xFF == ord('q'):  # cv2.waitKey - время задержки кадра
        break
# When everything is done, release the capture
video_capture.release()
out.release()
cv2.destroyAllWindows()

### Вариант дальнейшего внедрения модели на Raspberry Pi:
 Модель можно внедрить на базе Raspberry Pi, настроить систему
распознавания маски на лице с активацией зуммера + подключение к системе открытия двери. Во время пандемии коронавируса это бюджетное решение может пригодиться небольшим магазинам.
https://proglib.io/p/uchim-raspberry-pi-raspoznavat-nalichie-maski-na-lice-2020-11-24

### Вариант дальнейшего внедрения e-mail оповещения при срабатывании сигналов нарушения:
https://proglib.io/p/uchim-raspberry-pi-raspoznavaniyu-lic-2020-11-15